In [1]:
import osmnx as ox
import networkx as nx
import json
import numpy as np
import random

from ipywidgets import IntProgress
from IPython.display import display

In [2]:
# lib.graph.load
G = ox.save_load.load_graphml('../../prepare/data/city.graphml')

In [3]:
# lib.graph.nodes
allTagged = G.nodes(data="tag")

allHouses = []
allInfra = []

for (key, tag) in allTagged:
    # Check if key is even then add pair to new dictionary
    if tag == "apartments":
        allHouses.append(key)
    if tag == "fire_station":
        allInfra.append(key)
    if tag == "hospital":
        allInfra.append(key)
    if tag == "shop":
        allInfra.append(key)

In [32]:
# lib.paths

def route2Path(G, route):
    length = 0 # meters
    time = 0 # minutes

    for i in range(1, len(route)):
        edge = G[route[i - 1]][route[i]][0]
        length += edge['length']
    
    time = length / (speed / 3.6) / 60
    
    return {
        "route": route, 
        "length": length, 
        "time": time
    }

def getPath(G, start, end):     
    return route2Path(G, nx.dijkstra_path(G, start, end, "length"))

def joinPaths(a, b): 
    return {
        "route": a['route'] + b['route'],
        "length": a['length'] + b['length'],
        "time": a['time'] + b['time']
    }

In [33]:
# 1.1 params
speed = 40 # km/h

houses = random.sample(allHouses, 100)
infra = random.sample(allInfra, 10)

In [42]:
houseObjPaths = {}
objHousePaths = {}
roundPaths = {}

i = 0
progress = IntProgress(min=0, max=len(houses) + len(infra), value=0)
display(progress)

# Find all house -> obj paths
for house in houses:
    _, paths = nx.single_source_dijkstra(G, house, weight="length")
    
    progress.value += 1
    progress.description = f'{progress.value}/{progress.max}'
    
    houseObjPaths[house] = {}
    for obj in infra:
        houseObjPaths[house][obj] = route2Path(G, paths[obj])

# Find all obj -> house paths
for obj in infra:
    _, paths = nx.single_source_dijkstra(G, obj, weight="length")
    
    progress.value += 1
    progress.description = f'{progress.value}/{progress.max}'
    
    objHousePaths[obj] = {}   
    for house in houses:
        if(house in paths):
            objHousePaths[obj][house] = route2Path(G, paths[house])

# Make round paths form them
for house in houses:
    for obj in infra:
        if(obj in houseObjPaths[house]) and (house in objHousePaths[obj]):
            roundPaths[f"{house}-{obj}"] = joinPaths(houseObjPaths[house][obj], objHousePaths[obj][house])
            roundPaths[f"{obj}-{house}"] = joinPaths(objHousePaths[obj][house], houseObjPaths[house][obj])
        
progress.bar_style = "success" 

IntProgress(value=0, max=110)

In [43]:
houseClosestObj = {}
for house in houseObjPaths:
    shortestPath = {"length": float('inf')}
    closestObj = None
    for obj in houseObjPaths[house]:
        if(houseObjPaths[house][obj]['length'] < shortestPath['length']):
            shortestPath = houseObjPaths[house][obj]
            closestObj = obj
    
    houseClosestObj[house] = closestObj

In [44]:
objClosestHouse = {}
for obj in infra:
    shortestPath = {"length": float('inf')}
    closestHouse = None
    for house in objHousePaths[obj]:
        if(objHousePaths[obj][house]['length'] < shortestPath['length']):
            shortestPath = objHousePaths[obj][house]
            closestHouse = house
    
    objClosestHouse[obj] = closestHouse

In [47]:
shortestHouseObjRounds = {}
for house in houseObjPaths:
    shortestPath = {"length": float('inf')}
    closestObj = None
    for obj in houseObjPaths[house]:
        if (f'{house}-{obj}' in roundPaths) and (roundPaths[f'{house}-{obj}']['length'] < shortestPath['length']):
            shortestPath = roundPaths[f'{house}-{obj}']
            closestObj = obj
    
    shortestHouseObjRounds[house] = closestObj

In [49]:
# for house in houses:
#     obj = houseClosestObj[house]
#     ox.plot_graph_route(G, houseObjPaths[house][obj]['route'], edge_linewidth=0.1, node_size=0.2)